### Pokemon Prediction

In [2]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import seaborn as sns
import matplotlib.pyplot as plt

In [3]:
df = pd.read_csv('pokemon.csv')
df.head()

,Number,Name,Type 1,Type 2,Abilities,HP,Att,Def,Spa,Spd,...,Against Bug,Against Rock,Against Ghost,Against Dragon,Against Dark,Against Steel,Against Fairy,Height,Weight,BMI
0,1,Bulbasaur,Grass,Poison,"['Chlorophyll', 'Overgrow']",45,49,49,65,65,...,1.0,1.0,1.0,1.0,1.0,1.0,0.5,0.7,6.9,14.1
1,2,Ivysaur,Grass,Poison,"['Chlorophyll', 'Overgrow']",60,62,63,80,80,...,1.0,1.0,1.0,1.0,1.0,1.0,0.5,1.0,13.0,13.0
2,3,Venusaur,Grass,Poison,"['Chlorophyll', 'Overgrow']",80,82,83,100,100,...,1.0,1.0,1.0,1.0,1.0,1.0,0.5,2.0,100.0,25.0
3,3,Mega Venusaur,Grass,Poison,['Thick Fat'],80,100,123,122,120,...,1.0,1.0,1.0,1.0,1.0,1.0,0.5,2.4,155.5,27.0
4,4,Charmander,Fire,NaN,"['Blaze', 'Solar Power']",39,52,43,60,50,...,0.5,2.0,1.0,1.0,1.0,0.5,0.5,0.6,8.5,23.6


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1032 entries, 0 to 1031
Data columns (total 44 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Number                   1032 non-null   int64  
 1   Name                     1032 non-null   object 
 2   Type 1                   1032 non-null   object 
 3   Type 2                   548 non-null    object 
 4   Abilities                1032 non-null   object 
 5   HP                       1032 non-null   int64  
 6   Att                      1032 non-null   int64  
 7   Def                      1032 non-null   int64  
 8   Spa                      1032 non-null   int64  
 9   Spd                      1032 non-null   int64  
 10  Spe                      1032 non-null   int64  
 11  BST                      1032 non-null   int64  
 12  Mean                     1032 non-null   float64
 13  Standard Deviation       1032 non-null   float64
 14  Generation              

In [5]:
df.describe()

,Number,HP,Att,Def,Spa,Spd,Spe,BST,Mean,Standard Deviation,...,Against Bug,Against Rock,Against Ghost,Against Dragon,Against Dark,Against Steel,Against Fairy,Height,Weight,BMI
count,1032.000000,1032.000000,1032.000000,1032.000000,1032.000000,1032.000000,1032.000000,1032.000000,1032.000000,1032.000000,...,1032.000000,1032.000000,1032.000000,1032.000000,1032.000000,1032.000000,1032.000000,1032.000000,1032.000000,1032.000000
mean,439.226744,69.906008,80.526163,74.609496,72.918605,72.139535,68.548450,438.648256,73.108043,20.028104,...,1.002180,1.239826,1.025678,0.974806,1.074855,0.992006,1.094234,1.286822,71.879845,136.735756
std,261.871350,26.189155,32.542374,30.905972,32.773495,27.625876,30.219526,120.675545,20.112591,10.830298,...,0.613111,0.699361,0.577269,0.378040,0.475292,0.511859,0.535159,1.391501,132.872741,3111.666658
min,1.000000,1.000000,5.000000,5.000000,10.000000,20.000000,5.000000,175.000000,29.166667,0.000000,...,0.250000,0.250000,0.000000,0.000000,0.250000,0.250000,0.250000,0.100000,0.100000,0.000000
25%,211.750000,50.000000,55.000000,50.000000,50.000000,50.000000,45.000000,330.000000,55.000000,12.801910,...,0.500000,1.000000,1.000000,1.000000,1.000000,0.500000,1.000000,0.600000,9.000000,18.600000
50%,434.500000,67.000000,78.000000,70.000000,65.000000,70.000000,65.000000,459.000000,76.500000,18.484228,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,29.750000,28.350000
75%,667.250000,83.000000,100.000000,90.000000,95.000000,90.000000,90.000000,515.000000,85.833333,24.835709,...,1.000000,2.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.600000,71.275000,42.200000
max,898.000000,255.000000,190.000000,230.000000,194.000000,230.000000,200.000000,780.000000,130.000000,103.215659,...,4.000000,4.000000,4.000000,2.000000,4.000000,4.000000,4.000000,20.000000,999.900000,99990.000000


In [6]:
from sklearn.preprocessing import LabelEncoder
# Initialize LabelEncoder
le = LabelEncoder()
# Fit and transform the 'city' column
df['Type1_Numeric'] = le.fit_transform(df['Type 1'])
print(df['Type1_Numeric'])
 
 
# Fit and transform the 'city' column
df['Abilities_Numeric'] = le.fit_transform(df['Abilities'])
print(df['Abilities_Numeric'])
 
df_numeric = df.select_dtypes(include=[int, float])
 
df_numeric.corr().style.background_gradient(cmap='coolwarm', axis=None)
 

0        9
1        9
2        9
3        9
4        6
        ..
1027    11
1028     8
1029    14
1030    14
1031    14
Name: Type1_Numeric, Length: 1032, dtype: int32
0        93
1        93
2        93
3       582
4        67
       ... 
1027     78
1028    245
1029    590
1030     37
1031     37
Name: Abilities_Numeric, Length: 1032, dtype: int32


,Mean,Standard Deviation,Generation,Final Evolution,Legendary,Mega Evolution,Alolan Form,Galarian Form,Against Normal,Against Fire,Against Water,Against Electric,Against Grass,Against Ice,Against Fighting,Against Poison,Against Ground,Against Flying,Against Psychic,Against Bug,Against Rock,Against Ghost,Against Dragon,Against Dark,Against Steel,Against Fairy,Height,Weight,BMI,Type1_Numeric,Abilities_Numeric
Mean,1.000000,0.422135,0.173879,0.775027,0.526330,0.346077,-0.049442,-0.001863,-0.091032,-0.074386,0.012029,-0.036220,-0.040917,0.048197,0.007583,-0.093642,0.055863,-0.098393,-0.091069,-0.006508,-0.039485,0.151059,0.098818,0.087097,0.002667,0.122764,0.531366,0.478253,-0.011221,0.019126,0.091138
Standard Deviation,0.422135,1.000000,0.083804,0.337684,0.144894,0.245178,0.028326,0.007512,-0.242749,0.002712,0.164287,-0.048302,0.086025,-0.087334,0.105013,-0.201559,0.169976,-0.142761,-0.102125,-0.051731,-0.102292,0.124427,-0.068970,0.083296,0.109908,0.005951,0.303494,0.347870,0.075462,0.057697,0.084533
Generation,0.173879,0.083804,1.000000,0.075192,0.164046,0.156357,0.151207,0.221676,-0.145607,0.055202,-0.034793,-0.082537,-0.111657,0.000345,0.027956,0.022368,0.040752,-0.000152,-0.126365,0.009454,-0.035817,0.095784,-0.009933,0.022190,0.075183,0.151698,0.080206,0.126700,0.035759,-0.087156,0.005007
Final Evolution,0.775027,0.337684,0.075192,1.000000,0.289849,0.198024,-0.002517,-0.046766,-0.018645,-0.016608,-0.029712,0.014792,-0.031056,0.014210,0.007095,-0.034372,-0.004752,-0.041253,-0.065396,-0.003257,-0.002987,0.040750,0.008731,0.007651,-0.007027,0.087920,0.362825,0.312899,-0.036969,-0.011701,0.051505
Legendary,0.526330,0.144894,0.164046,0.289849,1.000000,0.026886,-0.049462,0.012442,-0.032863,-0.036439,-0.036245,-0.034940,-0.081271,0.047659,-0.064959,-0.042527,-0.000191,-0.051301,-0.072388,0.033815,-0.050891,0.179073,0.067981,0.136868,0.011606,0.099778,0.346454,0.398302,0.082908,0.002821,0.077320
Mega Evolution,0.346077,0.245178,0.156357,0.198024,0.026886,1.000000,-0.030064,-0.031722,-0.013706,-0.016695,0.019958,-0.023369,-0.024370,0.017677,-0.011794,-0.033720,0.006401,-0.000759,-0.040842,-0.013690,-0.019312,0.032976,0.021017,-0.021305,0.007936,0.065710,0.188389,0.151113,-0.007031,-0.016918,0.156814
Alolan Form,-0.049442,0.028326,0.151207,-0.002517,-0.049462,-0.030064,1.000000,-0.018730,-0.054363,0.056331,0.050546,-0.064954,-0.003508,-0.036936,0.167567,-0.057282,0.137870,-0.085400,-0.119268,0.032742,-0.029828,-0.031587,-0.030295,-0.044365,0.031018,0.011123,0.006584,-0.001340,-0.003390,-0.015726,0.046282
Galarian Form,-0.001863,0.007512,0.221676,-0.046766,0.012442,-0.031722,-0.018730,1.000000,-0.008539,0.020576,0.020537,0.000933,-0.029273,-0.027402,0.020940,-0.030687,0.003988,-0.028904,-0.042308,0.005235,0.009588,0.072928,-0.046434,0.051829,0.057153,0.001515,-0.004733,-0.026341,-0.004383,0.010844,-0.028556
Against Normal,-0.091032,-0.242749,-0.145607,-0.018645,-0.032863,-0.013706,-0.054363,-0.008539,1.000000,-0.073635,-0.212624,0.091445,-0.085244,0.079382,0.050856,0.482139,-0.268113,0.244339,0.117092,0.250121,0.184964,-0.431523,0.119605,-0.439284,-0.086122,0.154994,-0.044041,-0.124895,0.011626,-0.101222,-0.055803
Against Fire,-0.074386,0.002712,0.055202,-0.016608,-0.036439,-0.016695,0.056331,0.020576,-0.073635,1.000000,-0.223703,-0.251996,-0.441270,0.099081,-0.012401,0.120430,-0.226920,0.455673,-0.041413,0.161669,0.117959,0.048593,-0.281364,0.016357,0.120562,-0.199075,-0.092088,-0.016429,-0.005799,-0.282642,-0.023370


### ANN - Multi Class

In [27]:
from sklearn.model_selection import train_test_split

# Define all Against Columns
against_columns = [col for col in df.columns if 'Against' in col]

# X = df.iloc[:, 23:41].to_numpy()
X = df[against_columns].to_numpy()
Y = df['Type1_Numeric'].to_numpy()

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

# Transform to tensors
X_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.long)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.float32)

class PokemonANN(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(in_features=18, out_features=256)
        self.fc2 = nn.Linear(in_features=256, out_features=128)
        self.output = nn.Linear(in_features=128, out_features=18)
        
    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.output(x)
        return x

model = PokemonANN()

loss_function = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

epochs = 100
for epoch in range(epochs):
    optimizer.zero_grad()
    y_hat = model.forward(X_train)
    loss = loss_function(y_hat, y_train)
    loss.backward()
    optimizer.step()
    if epoch % 10 == 0:
        print(f'Epoch: {epoch} Loss: {loss}')

Epoch: 0 Loss: 2.8979735374450684
Epoch: 10 Loss: 0.4327313005924225
Epoch: 20 Loss: 0.23636062443256378
Epoch: 30 Loss: 0.15220056474208832
Epoch: 40 Loss: 0.1355365365743637
Epoch: 50 Loss: 0.1276998668909073
Epoch: 60 Loss: 0.12897811830043793
Epoch: 70 Loss: 0.12327411770820618
Epoch: 80 Loss: 0.1231604591012001
Epoch: 90 Loss: 0.12065296620130539
